# BHS 2020 project: Brain Learning Unicorn Project

*Using UKBB to apply the new tools learned at the BHS*

Author: Elise Douard

## Summary

*Can a model predict if an individual is carrier of a pathogenic genetic copy number variant based on strucftural brain imagery derived data?*

This project aims to feed a learning model with brains to predict if an individual is carrier of a pathogenic genetic variant (meaning that the DNA alteration is deleterious and formally associated to neurodevelopmental disorders and other psychiatric disorders).

The hypothesis is not based on strong assumptions (even if there is multiple publications showing common brain alterations associated to pathogenic CNVs), the focus will be on learning how to apply machine learning model for multimodal dataset (derived anatomical MRI data, genetic, other clinical data).

## Background

### What is a pathogenic genetic Copy Number Variants (CNV)?

A pathogenic CNV is a gain or a loss of a DNA portion, which encompassed genes essential for the normal development of an individual, and which is formally associated to neurodevelopmental disorders or psychiatric disorders.

## Loading all the libraries and datasets

In [37]:
# Import all the libraries of interest
import os
import pandas
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

from scipy import stats
from ipywidgets import widgets
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

%matplotlib inline

In [2]:
# Import my left and right hemisphers dataset
UKBB_gen_MRI_path_lh = ('/home/elise/Documents/UKBB/FreeSurfer_Desikan_parcellation/Freesurfer_aparc_volume_lh_UKBB_40k_CNVs_apr2020.csv')
UKBB_gen_MRI_lh = pandas.read_csv(UKBB_gen_MRI_path_lh,sep=',')

UKBB_gen_MRI_path_rh = ('/home/elise/Documents/UKBB/FreeSurfer_Desikan_parcellation/Freesurfer_aparc_volume_rh_UKBB_40k_CNVs_apr2020.csv')
UKBB_gen_MRI_rh = pandas.read_csv(UKBB_gen_MRI_path_rh,sep=',')
# The total volume is duplicated in the lh and rh files
UKBB_gen_MRI_rh = UKBB_gen_MRI_rh.rename(columns={'Total': 'Total_bis'})

# Import the file with Total Intracranial Volume (TIV)
UKBB_TIV_MRI_path = ('/home/elise/Documents/UKBB/FreeSurfer_Desikan_parcellation/ASEG_UKBB_40k_SubCortVolumes_and_Globals.csv')
UKBB_TIV_MRI = pandas.read_csv(UKBB_TIV_MRI_path,sep=',')
UKBB_TIV_MRI = UKBB_TIV_MRI[["eid","EstimatedTotalIntraCranialVol"]]

I preselected the most pathogenic CNVs among the list of recurrent CNVs from Kendall et al. 

I selected the CNVs with the highest score of probability to be intolerant to the loss of function, which means that if the genes encompassed by the CNVs are altered, it will be super rare, reflecting the highly deleterious effect on the individual.

This list will be used to select the carriers that will be compared to the controls.

The other carriers detected by Kendall have only low penetrance recurrent CNV, with lower impact. 

In [3]:
# Import my list of pathogenic CNVs (Later)


In [4]:
# Merge both datasets (previously remove the duplicated columns in the rh dataset)
UKBB_gen_MRI_rh = UKBB_gen_MRI_rh.drop(["sex", "site", "age", "KirovPathCNV","Total_bis"], axis=1)
UKBB_gen_MRI = pandas.merge(UKBB_gen_MRI_lh,UKBB_gen_MRI_rh,on="eid")
UKBB_gen_MRI = pandas.merge(UKBB_TIV_MRI,UKBB_gen_MRI,on="eid")

## Preliminary description of the dataset + visualization

### Creating the dataset used for the analyses

These are the raw data from which I will extract my groups of CNV carriers and controls. 

In [5]:
# Little overview of my data
UKBB_gen_MRI.head()

,eid,EstimatedTotalIntraCranialVol,sex,site,age,KirovPathCNV,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,...,rh_rostralanteriorcingulate_volume,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
0,2316851,1.748858e+06,Male,11025,68.172043,control,2734,1958,7753,3771,...,2194,18139,23826,15032,12618,12735,1011,2700,897,7054
1,2290919,1.481886e+06,Female,11025,62.032258,control,2350,1764,7733,3803,...,2740,15966,24448,14464,12152,9418,1036,2664,1004,7464
2,2817406,1.531335e+06,Female,11027,51.844086,control,3058,1974,6856,2328,...,2132,16940,25453,15057,15230,14980,1242,3024,1408,8115
3,5428153,1.769562e+06,Male,11025,53.516129,control,2733,1183,7594,2934,...,2352,18350,23843,15003,13481,10887,1090,2421,1044,7514
4,5578868,1.598378e+06,Male,11025,73.295699,control,2448,1838,5536,4099,...,1911,13848,19993,13400,11949,11077,1167,2696,1061,6570


In [6]:
UKBB_gen_MRI.describe()

,eid,EstimatedTotalIntraCranialVol,site,age,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,lh_fusiform_volume,...,rh_rostralanteriorcingulate_volume,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_frontalpole_volume,rh_temporalpole_volume,rh_transversetemporal_volume,rh_insula_volume
count,3.575900e+04,3.575900e+04,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,...,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000,35759.000000
mean,3.519977e+06,1.548799e+06,11025.629044,64.094198,2501.048268,1721.925082,6662.638720,3154.962779,1974.154199,10408.102268,...,2163.375010,16411.243323,22660.586650,13841.935485,12161.835454,10872.072150,1175.737940,2565.871557,990.818591,7396.985822
std,1.450640e+06,1.514822e+05,0.855953,7.546142,455.873634,534.325934,1085.058175,567.506845,398.354431,1333.242227,...,464.115517,2354.064924,2863.884835,1788.206422,1462.044236,1582.462572,169.410579,364.759625,175.456664,879.537386
min,1.000169e+06,6.431333e+05,11025.000000,44.454301,1133.000000,253.000000,3090.000000,647.000000,572.000000,4929.000000,...,270.000000,9187.000000,12479.000000,6749.000000,6919.000000,5337.000000,441.000000,366.000000,401.000000,4606.000000
25%,2.265808e+06,1.440705e+06,11025.000000,58.251344,2186.000000,1350.000000,5902.000000,2752.000000,1701.000000,9483.000000,...,1847.000000,14741.500000,20634.000000,12609.000000,11141.000000,9766.000000,1060.000000,2321.000000,868.000000,6763.000000
50%,3.519394e+06,1.541247e+06,11025.000000,64.602151,2460.000000,1696.000000,6589.000000,3099.000000,1943.000000,10336.000000,...,2148.000000,16231.000000,22439.000000,13766.000000,12087.000000,10767.000000,1166.000000,2552.000000,973.000000,7326.000000
75%,4.773582e+06,1.651558e+06,11026.000000,69.994624,2768.000000,2060.000000,7342.000000,3498.000000,2205.000000,11260.000000,...,2463.000000,17884.000000,24481.000000,15005.000000,13099.500000,11887.000000,1281.000000,2795.000000,1097.000000,7958.000000
max,6.025437e+06,2.227009e+06,11027.000000,82.040323,5521.000000,7040.000000,12622.000000,6358.000000,4999.000000,17952.000000,...,4368.000000,28318.000000,38254.000000,21924.000000,19474.000000,18373.000000,2388.000000,4601.000000,1944.000000,11642.000000


In [7]:
# Change my "site" column into categorical variable 
UKBB_gen_MRI['site'] = UKBB_gen_MRI['site'].astype(object)
# Show the description of the columns with categorie variables
UKBB_gen_MRI.describe(include = 'object')

,sex,site,KirovPathCNV
count,35759,35759,35759
unique,2,3,65
top,Female,11025,control
freq,18951,22192,34494


Kendall and Kirov et al. (2019) have already annotated the individuals carriers of at least on known recurrent CNV ("KirovPathCNV" column). 
**BUT** the CNVs used in their list are not all pathogenic - meaning they are not all fully penetrant/highly associated to psychiatric disorders.

In [8]:
UKBB_gen_MRI['KirovPathCNV'].value_counts()

control                                              34494
15q13.3dup(CHRNA7)                                     235
2q13del(NPHP1)                                         223
2q13dup(NPHP1)                                         189
15q11.2dup                                             147
                                                     ...  
15q13.3dup(CHRNA7) TAR_dup                               1
15q13.3dup(CHRNA7) 15q11.2dup                            1
17p13.3(YWHAE)del                                        1
15q13.3dup(CHRNA7) 15q11q13dup_BP3-BP4(APBA2,TJP)        1
15q13.3dup(CHRNA7) 2q13dup(NPHP1)                        1
Name: KirovPathCNV, Length: 65, dtype: int64

There is a total of 1,265 individuals with a potentially pathogenic CNV. 

I want to select only the ones from my list of pathogenic CNVs.

In [9]:
# Later

In [10]:
# Create the binary groupe variable
UKBB_gen_MRI["group"] = np.where(UKBB_gen_MRI.KirovPathCNV == "control", "control", "carrier")
UKBB_gen_MRI['group'].value_counts()

control    34494
carrier     1265
Name: group, dtype: int64

### Visualization of the dataset content

## Preliminary analyses + visualization

Numerous papers highlighted a consistent alteration of the insula in the carriers of higly pathogenic CNVs and more widely, in individuals with psychiatric disorders.

Let's see if we observe a difference in insula volume between groups at first sight.

1) First, let's create the volumes rescaled in function of the TIV and these z-scored volumes based on the controls mean and sd volumes 

In [11]:
# Subset of controls 
UKBB_gen_MRI_controls = UKBB_gen_MRI[UKBB_gen_MRI.KirovPathCNV == 'control']

In [12]:
# Create the TIV rescaled volumes columns
cols = list(UKBB_gen_MRI.columns)
cols = [e for e in cols if e not in ("eid", "EstimatedTotalIntraCranialVol", "sex", "site", "age", "KirovPathCNV","group")]
#cols
for col in cols:
    # For the scores rescaled in function of the TIV
    col_TIVres = col + '_TIVres'
    UKBB_gen_MRI[col_TIVres] = UKBB_gen_MRI[col]/UKBB_gen_MRI["EstimatedTotalIntraCranialVol"]
    # For the z-scored transformation of these new scores in function of the controls mean and sd volumes.
    col_zscore = col + '_zscore'
    UKBB_gen_MRI[col_zscore] = (UKBB_gen_MRI[col] - UKBB_gen_MRI_controls[col].mean())/UKBB_gen_MRI_controls[col].std(ddof=0)

In [13]:
UKBB_gen_MRI.head()

,eid,EstimatedTotalIntraCranialVol,sex,site,age,KirovPathCNV,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,...,rh_supramarginal_volume_TIVres,rh_supramarginal_volume_zscore,rh_frontalpole_volume_TIVres,rh_frontalpole_volume_zscore,rh_temporalpole_volume_TIVres,rh_temporalpole_volume_zscore,rh_transversetemporal_volume_TIVres,rh_transversetemporal_volume_zscore,rh_insula_volume_TIVres,rh_insula_volume_zscore
0,2316851,1.748858e+06,Male,11025,68.172043,control,2734,1958,7753,3771,...,0.007282,1.175042,0.000578,-0.973303,0.001544,0.366187,0.000513,-0.534288,0.004033,-0.392346
1,2290919,1.481886e+06,Female,11025,62.032258,control,2350,1764,7733,3803,...,0.006355,-0.919091,0.000699,-0.825793,0.001798,0.267560,0.000678,0.075529,0.005037,0.074183
2,2817406,1.531335e+06,Female,11027,51.844086,control,3058,1974,6856,2328,...,0.009782,2.592386,0.000811,0.389690,0.001975,1.253828,0.000919,2.378020,0.005299,0.814939
3,5428153,1.769562e+06,Male,11025,53.516129,control,2733,1183,7594,2934,...,0.006152,0.008338,0.000616,-0.507171,0.001368,-0.398171,0.000590,0.303499,0.004246,0.131077
4,5578868,1.598378e+06,Male,11025,73.295699,control,2448,1838,5536,4099,...,0.006930,0.128291,0.000730,-0.052840,0.001687,0.355228,0.000664,0.400386,0.004110,-0.943077


In [14]:
# Verifying that there is no NaN
UKBB_gen_MRI["lh_insula_volume_TIVres"].describe()

count    35759.000000
mean         0.004736
std          0.000424
min          0.003043
25%          0.004450
50%          0.004710
75%          0.004993
max          0.012113
Name: lh_insula_volume_TIVres, dtype: float64

2) Overview of distribution of insula volumes in the two groups (lh and rh separately)

In [22]:
# Reshape the dataset for the insula only
df_plots = UKBB_gen_MRI.melt(id_vars=["eid",'group',"sex", "site", "age", "KirovPathCNV"])
# Creating the column with the corresponding hemisphere 
df_plots["hemisphere"] = np.where(df_plots.variable.str.contains("lh"), "Left", np.where(df_plots.variable.str.contains("rh"), "Right", "Global"))
# Creating the column with the corresponding transformation
df_plots["transformation"] = np.where(df_plots.variable.str.contains("_TIVres"), "rescaled with TIV", np.where(df_plots.variable.str.contains("_zscore"), "z-scored", "None"))
# Remove these informations from the variable column
df_plots['variable'] = df_plots['variable'].replace(["lh_","rh_","_TIVres","_zscore", "_volume"], '', regex=True)

df_plots = df_plots.rename(columns={'variable': 'region'})

df_plots.describe()

,eid,age,value
count,7.437872e+06,7.437872e+06,7.437872e+06
mean,3.519977e+06,6.409420e+01,1.728720e+04
std,1.450619e+06,7.546037e+00,1.517049e+05
min,1.000169e+06,4.445430e+01,-6.028120e+00
25%,2.265743e+06,5.825000e+01,1.618432e-03
50%,3.519394e+06,6.460215e+01,1.536231e-02
75%,4.773661e+06,6.999462e+01,2.671000e+03
max,6.025437e+06,8.204032e+01,2.227009e+06


In [23]:
df_plots.describe(include = 'object')

,group,sex,site,KirovPathCNV,region,hemisphere,transformation
count,7437872,7437872,7437872,7437872,7437872,7437872,7437872
unique,2,2,3,65,36,3,3
top,control,Female,11025,control,superiorfrontal,Left,None
freq,7174752,3941808,4615936,7174752,214554,3647418,2503130


In [ ]:
fig.add_trace(go.Heatmap(z=df.values.tolist(), colorscale="Viridis"))

# Update plot sizing
fig.update_layout(
    width=800,
    height=900,
    autosize=False,
    margin=dict(t=100, b=0, l=0, r=0),
)

# Update 3D scene options
fig.update_scenes(
    aspectratio=dict(x=1, y=1, z=0.7),
    aspectmode="manual"
)

# Add drowdowns
# button_layer_1_height = 1.08
button_layer_1_height = 1.12
button_layer_2_height = 1.065

fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["colorscale", "Viridis"],
                    label="Viridis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Cividis"],
                    label="Cividis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Blues"],
                    label="Blues",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Greens"],
                    label="Greens",
                    method="restyle"
                ),
            ]),
            type = "buttons",
            direction="right",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=["reversescale", False],
                    label="False",
                    method="restyle"
                ),
                dict(
                    args=["reversescale", True],
                    label="True",
                    method="restyle"
                )
            ]),
            type = "buttons",
            direction="right",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.13,
            xanchor="left",
            y=button_layer_2_height,
            yanchor="top"
        ),
        dict(
            buttons=list([
                dict(
                    args=[{"contours.showlines": False, "type": "contour"}],
                    label="Hide lines",
                    method="restyle"
                ),
                dict(
                    args=[{"contours.showlines": True, "type": "contour"}],
                    label="Show lines",
                    method="restyle"
                ),
            ]),
            type = "buttons",
            direction="right",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.5,
            xanchor="left",
            y=button_layer_2_height,
            yanchor="top"
        ),
    ]
)

fig.update_layout(
    annotations=[
        dict(text="colorscale", x=0, xref="paper", y=1.1, yref="paper",
                             align="left", showarrow=False),
        dict(text="Reverse<br>Colorscale", x=0, xref="paper", y=1.06,
                             yref="paper", showarrow=False),
        dict(text="Lines", x=0.47, xref="paper", y=1.045, yref="paper",
                             showarrow=False)
    ])

fig.show()

In [47]:
# Make an interractive qqplot to see the distribution of the volume in the insula 
fig2 = go.FigureWidget()

# Choices 3: Left or Right hemisphere



qq = stats.probplot(np.random.lognormal(df_plots["value"][(df_plots.transformation== "rescaled with TIV") & 
                                                  (df_plots['region'] == 'insula') & 
                                                  (df_plots['hemisphere'] == 'Left')])
                    , dist='lognorm', sparams=(1))

x = np.array([qq[0][0][0],qq[0][0][-1]])
pts = go.Scatter(x=qq[0][0],
                 y=qq[0][1], 
                 mode = 'markers',
                 showlegend=False
                )
line = go.Scatter(x=x,
                  y=qq[1][1] + qq[1][0]*x,
                  showlegend=False,
                  mode='lines'
                 )

data = [pts, line]
layout = dict(xaxis = dict(zeroline = False,
                           linewidth = 1,
                           mirror = True),
              yaxis = dict(zeroline = False, 
                           linewidth = 1,
                           mirror = True),
             )

fig2 = dict(data=data, layout=layout)

def response(change):
        filter_list = [i and j for i, j in
                           zip(df_plots['region'] == region.value,
                               df_plots['transformation'] == transform.value)]
        temp_df = df_plots[filter_list]

        x1 = temp_df['hemisphere'][(temp_df['group'] == 'control')]
        x2 = temp_df['hemisphere'][(temp_df['group'] == 'carrier')]
        y1 = temp_df["value"][(temp_df['group'] == 'control')]
        y2 = temp_df["value"][(temp_df['group'] == 'carrier')]
        with fig.batch_update():
            fig.data[0].x = x1
            fig.data[0].y = y1
            fig.data[1].x = x2
            fig.data[1].y = y2
            fig.update_layout(violinmode='group')
# iplot(fig, show_link=False)
region.observe(response, names="value")
transform.observe(response, names="value")

In [51]:
# Assign the default figure widget with two traces
fig3 = go.FigureWidget()

# Choices 1: Rescale with TIV only or also with z-score
transform = widgets.Dropdown(
    options=list(df_plots['transformation'].unique()),
    value='rescaled with TIV',
    description='Transformation applied to the volumes:',
)

# Choices 2: Chose the region of interest
region = widgets.Dropdown(
    options=list(df_plots['region'].unique()),
    value='insula',
    description='Region of interest:',
)

# Create the violin plots with the default values
fig3.add_trace(go.Violin(x=df_plots['hemisphere'][(df_plots['group'] == 'control') & 
                                                  (df_plots.transformation== "rescaled with TIV") & 
                                                  (df_plots['region'] == 'insula')],
                        y=df_plots["value"][(df_plots['group'] == 'control') & 
                                                  (df_plots.transformation== "rescaled with TIV") & 
                                                  (df_plots['region'] == 'insula')],
                        legendgroup='controls', scalegroup='controls', name='Controls',
                        line_color='blue', box_visible=True, meanline_visible=True)
             )
fig3.add_trace(go.Violin(x=df_plots['hemisphere'][(df_plots['group'] == 'carrier') & 
                                                  (df_plots.transformation== "rescaled with TIV") & 
                                                  (df_plots['region'] == 'insula')],
                        y=df_plots["value"][(df_plots['group'] == 'carrier') & 
                                                  (df_plots.transformation== "rescaled with TIV") & 
                                                  (df_plots['region'] == 'insula')],
                        legendgroup='carriers', scalegroup='carriers', name='Carriers',
                        line_color='orange', box_visible=True, meanline_visible=True)
             )

fig3.update_layout(violinmode='group')
fig3.layout.xaxis.title = 'Hemisphere'
fig3.layout.yaxis.title = 'Volume'
fig3.layout.title = 'Distribution of the brain region volumes from UKBioBank controls and carriers'

# Add the conditions for the widget entry and refresh the plots
def response(change):
        filter_list = [i and j for i, j in
                           zip(df_plots['region'] == region.value,
                               df_plots['transformation'] == transform.value)]
        temp_df = df_plots[filter_list]

        x1 = temp_df['hemisphere'][(temp_df['group'] == 'control')]
        x2 = temp_df['hemisphere'][(temp_df['group'] == 'carrier')]
        y1 = temp_df["value"][(temp_df['group'] == 'control')]
        y2 = temp_df["value"][(temp_df['group'] == 'carrier')]
        with fig3.batch_update():
            fig3.data[0].x = x1
            fig3.data[0].y = y1
            fig3.data[1].x = x2
            fig3.data[1].y = y2
            fig3.update_layout(violinmode='group')


region.observe(response, names="value")
transform.observe(response, names="value")

In [52]:
container = widgets.HBox([region, transform])
widgets.VBox([container, fig3])

In [ ]:
import plotly.io as pio
pio.write_html(fig, file=’index.html’, auto_open=True)